In [ ]:
import quest
from paramnb import Widgets
from pprint import pprint
import warnings
warnings.simplefilter('ignore')

from plotly.offline import init_notebook_mode
from plotly.offline import iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

# Query Quest for Available Data Providers

In [ ]:
quest_providers = quest.api.get_providers()
pprint(quest_providers)
quest_publishers = quest.api.get_publishers()
quest_publishers

In [ ]:
quest_services = quest.api.get_services()
pprint(quest_services)
print("\nNumber of services: ", len(quest_services))
print("The one we want: ", quest_services[-1])

# Query USGS Data Service for Data

In [ ]:
# nwis_s_features = quest.api.get_features(quest_services[-1], filters={'bbox': [-91, 32.3, -90.8, 32.34]})
cuahsi_features = quest.api.get_features(quest_services[0])
# bbox ['long min', 'lat min', 'long max', 'lat max']
# nwis_s_features
cuahsi_features

In [ ]:
# collection_name = 'nwis_demo'
collection_name = 'the_demo2'
if collection_name in quest.api.get_collections():
    pass
else:
    quest.api.new_collection(collection_name)

# collection_feature = quest.api.add_features('nwis_demo', nwis_s_features[0])
collection_feature = quest.api.add_features('the_demo2', cuahsi_features[0])
collection_feature

In [ ]:
download_options = quest.api.download_options(collection_feature, fmt='param')[collection_feature[0]]
download_options

In [ ]:
Widgets(download_options)

In [ ]:
options = {'parameter':download_options.parameter, 
           'start': download_options.start, 
           'end': download_options.end, 
           'period': download_options.period
          }

In [ ]:
# staged_id = quest.api.stage_for_download(collection_feature, options=options)
staged_id = quest.api.stage_for_download(collection_feature, options=None)
staged_id

In [ ]:
quest.api.download_datasets(staged_id)

In [ ]:
df = quest.api.open_dataset(staged_id[0])
df.head()

In [ ]:
parameter = df.metadata['parameter']
data = [go.Scatter(
          x=df.index,
          y=df[parameter],
          fill='tozeroy'
    )]

iplot(data)

# Apply a Filter to Transform Downloaded 

In [ ]:
quest.api.get_filters(filters={'dataset': staged_id[0]})

In [ ]:
filter_name = 'ts-resample'
filter_options = quest.api.apply_filter_options(filter_name, fmt='param')

In [ ]:
Widgets(filter_options)

In [ ]:
options = {'dataset':filter_options.dataset, 
           'period': filter_options.period, 
           'method': filter_options.method, 
          }

In [ ]:
new_dataset = quest.api.apply_filter(filter_name, options=options)['datasets'][0]

In [ ]:
df = quest.api.open_dataset(new_dataset)
df['date'] = df.index.to_timestamp()
df.head()

In [ ]:
parameter = df.metadata['parameter']
data = [go.Scatter(
          x=df['date'],
          y=df[parameter],
          fill='tozeroy'
    )]

iplot(data)

# Publish New Dataset to Data Depot

In [ ]:
quest.api.authenticate_provider('cuahsi-hydroshare')

In [ ]:
publish_options = quest.api.get_publish_options('pub://cuahsi-hydroshare:hs_pub', fmt='param')['pub://cuahsi-hydroshare:hs_pub']

In [ ]:
Widgets(publish_options)

In [ ]:
new_publish_options = {'title': publish_options.title, 
                       'abstract': publish_options.abstract, 
                       'keywords': publish_options.keywords, 
                       'dataset': new_dataset}

In [ ]:
quest.api.publish('pub://cuahsi-hs:hs_normal', options=new_publish_options)